[View in Colaboratory](https://colab.research.google.com/github/rdenadai/jupyter-pool/blob/master/nomes.ipynb)

## Nomes

Notebook que irá coletar todos os nomes de pessoas existentes no site [Dicionario de Nomes Próprios](https://www.dicionariodenomesproprios.com.br/).

O programa abaixo recebe como parâmetro o tamanho do nome que se quer apresentar, por padrão, só retorna nomes com quantidade de letras igual a 7. Por exemplo: Eduardo, Alberto.

In [6]:
!apt-get install libxml2-dev libxslt-dev
!pip install aiomultiprocess aiohttp cchardet aiodns numba lxml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libxslt1-dev' instead of 'libxslt-dev'
libxslt1-dev is already the newest version (1.1.29-2.1ubuntu1).
libxml2-dev is already the newest version (2.9.4+dfsg1-4ubuntu1.2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [0]:
import string
import asyncio
import functools
import concurrent.futures
from unicodedata import normalize
from collections import namedtuple
from bs4 import BeautifulSoup as bs
import aiohttp
from aiomultiprocess import Pool
from numba import jit
import numpy as np
import pandas as pd 

In [0]:
# Nome = namedtuple('Nome', ['href', 'nome'])
Letter = namedtuple('Letter', ['num', 'letter'])


def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')


async def fetch(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            # print(url)
            return await resp.text()


async def start(urls):
    responses = []
    async with Pool() as pool:
        responses = await pool.map(fetch, urls)
    return responses


def salvar_nomes(filename, nomes):
    with open(filename, 'w') as h:
        for nome in nomes:
            h.write(f'{nome}\n')


def parse_names(response, name_size=7):
    data = set()
    parser = bs(response, 'lxml')
    links = parser.find_all('a', class_='lista-nome')
    for link in links:
        # data.append(Nome(href=link['href'], nome=link.string))
        nome = link.string.strip()
        if len(nome) == name_size:
            data.add(nome)
    # Principal names!
    ols = parser.find_all('ol', class_='top-list')
    for ol in ols:
        links = ol.select('li a')
        for link in links:
            nome = link.string.strip()
            if len(nome) == name_size:
                data.add(nome)
    return '|'.join(list(data))


@jit(parallel=True)
def name_split(stringNomes):
    print('Realizando split dos nomes...')
    nomes = np.array([])
    for lre in stringNomes:
        nomes = np.concatenate((nomes, lre.split('|')), axis=None)
    yield filter(None, list(set(nomes.tolist())))
    

def retorna_nomes(sexo='M', name_size=7, pages=150):
    nomes = []
    try:
        if sexo.upper() == 'M':
            urls = [f'https://www.dicionariodenomesproprios.com.br/nomes-masculinos/{i}' for i in range(1, pages)]
        elif sexo.upper() == 'F':
            urls = [f'https://www.dicionariodenomesproprios.com.br/nomes-femininos/{i}' for i in range(1, pages)]
        loop = asyncio.get_event_loop()
        responses = loop.run_until_complete(asyncio.gather(start(urls)))[0]
        print('Terminado o download das páginas...')
        with concurrent.futures.ProcessPoolExecutor(max_workers=4) as exc:
            print('Iniciando parser das páginas...')
            result = exc.map(functools.partial(parse_names, name_size=name_size), responses, chunksize=5)
            print('Extraindo nomes do resultado...')
            nomes = name_split(result)
    except KeyboardInterrupt:
        loop.stop()
    return sorted(list(nomes)[0])


def side_by_side(M, N):
    return [M[i:i+N] for i in range(0, len(M), N)]

In [65]:
pages = 200
print('-' * 20)
print('Executando rotina (sexo masculino)...')
M = retorna_nomes(sexo='M', pages=pages)
print('\n')
print('-' * 20)
print('Executando rotina (sexo feminino)...')
F = retorna_nomes(sexo='F', pages=pages)

--------------------
Executando rotina (sexo masculino)...
Terminado o download das páginas...
Iniciando parser das páginas...
Extraindo nomes do resultado...
Realizando split dos nomes...


--------------------
Executando rotina (sexo feminino)...
Terminado o download das páginas...
Iniciando parser das páginas...
Extraindo nomes do resultado...
Realizando split dos nomes...


In [66]:
N = 20
print('Apresentando resultados...')
print('-' * 20)
print('Nomes Masculinos:')
df = pd.DataFrame(side_by_side(M, N), columns=['' for _ in range(N)])
df[df.isnull()] = ''
display(df)
print('\n')
print('Nomes Femininos:')
df = pd.DataFrame(side_by_side(F, N), columns=['' for _ in range(N)])
df[df.isnull()] = ''
display(df)
# print('Salvar arquivos...')
# print('Salvando nomes masculinos...')
# salvar_nomes('nomes_masculinos.txt', M)
# print('Salvando nomes femininos...')
# salvar_nomes('nomes_femininos.txt', F)

Apresentando resultados...
--------------------
Nomes Masculinos:


,,,,,,,,,,,,,,,,,,,,
0,Absalom,Absirto,Abúndio,Accioli,Acelino,Acemiro,Achille,Acilino,Acineto,Acrísio,Acúrcio,Acúrsio,Adalard,Adaliah,Adalton,Adelbar,Adelino,Adelmar,Adelpho,Adelson
1,Aderbal,Aderico,Adonias,Adriano,Adílson,Afrânio,Agasias,Aglaupo,Agledes,Agnaldo,Agrécio,Aguilar,Aguimar,Aguirre,Agustin,Aimberê,Ajagunã,Aladino,Alapaki,Alarico
2,Alberto,Alboino,Albílio,Alceste,Alcindo,Alcídes,Aldonca,Aleardo,Alebion,Alencar,Alexino,Alfonso,Alfredo,Alienor,Allyson,Almasor,Almeida,Almério,Aloísio,Aluísio
3,Amintas,Amintor,Amonati,Amorreu,Ampélio,Amâncio,Amândio,Américo,Amílcar,Ananias,Anatole,Andomar,Andrade,Andrzej,Aniceto,Ansaldo,Anselmo,Antelmo,Antenor,Anthony
4,Antoine,Antonin,Antunes,Antêmio,Antídio,Antíoco,Antíope,Antítio,Antônio,Apoinar,Apolino,Apollon,Aprígio,Aquiles,Aquilão,Aquílio,Arajuba,Arantes,Araquém,Araripe
5,Araruna,Aratama,Arcádio,Arduíno,Argaios,Arionte,Ariosto,Aristeu,Arlindo,Armando,Armindo,Armênio,Armínio,Arnaldo,Arnoldo,Arnolfo,Arnulfo,Arnóbio,Arouche,Arsênio
6,Artêmio,Arvídio,Aríston,Ascânio,Asenate,Aspásio,Assuero,Astride,Astêmio,Ataulfo,Augusto,Aurélio,Avelino,Azarias,Azevedo,Bacelar,Bacuara,Baiardo,Balbino,Baldwin
7,Balster,Barnaby,Barnabé,Barreto,Barsaré,Bartolo,Baruque,Basileu,Basílio,Batimeu,Batista,Beaulah,Belmiro,Belquir,Belquis,Beltrão,Ben-Hur,Benigno,Benício,Beraldo
8,Berardo,Bertino,Bertoli,Bertram,Berílio,Birigui,Bismark,Bogdana,Bogumil,Boiardo,Bolívar,Bourbon,Brandon,Brandão,Brendon,Briaréu,Bristol,Brizola,Bronwen,Brígido
9,Burnier,Caetano,Calisto,Calixto,Calvino,Calídno,Camarão,Canindé,Cardoso,Carioca,Carlito,Carlton,Carmelo,Caronte,Casimir,Castiel,Cataldi,Caxambu,Cecílio,Cercion




Nomes Femininos:


,,,,,,,,,,,,,,,,,,,,
0,Aaliyah,Aaminah,Abayomi,Abigail,Acemira,Acilina,Acirema,Adalina,Adamina,Adelina,Adeline,Adelita,Adozina,Adriana,Adriane,Adriele,Adônica,Agenora,Akeneki,Alarice
1,Alawara,Alberta,Alcinda,Alcione,Alcmena,Alcídia,Aldaíra,Alegria,Alekena,Alekona,Aleluia,Alenica,Alepeka,Alessia,Alethea,Alethia,Alisson,Allegra,Allende,Alméria
2,Althaia,Amalina,Amanacy,Amanara,Amapola,Ambrose,Amábile,América,Anabela,Analisa,Andresa,Andreza,Andrija,Andréia,Anelise,Anemone,Angícia,Anninka,Anselma,Anselme
3,Antelma,Antônia,Anêmola,Anúncia,Apolina,Apoédia,Arabela,Araceli,Aracema,Aretusa,Ariadne,Arianna,Arianne,Aricema,Aricina,Arienne,Aritana,Arlinda,Arminda,Arriaga
4,Arrieta,Arriola,Arsione,Artêmia,Ashiley,Aspásia,Assunta,Astéria,Ataléia,Augusta,Aurélia,Avelina,Azucena,Açucena,Baepeba,Balbina,Bambina,Banábil,Baraúna,Barbela
5,Barbosa,Bartira,Batilde,Beatriz,Bedelia,Belfort,Belinda,Belmira,Bendita,Benigna,Benilda,Benoite,Benícia,Berdine,Bernice,Bersabé,Bertina,Bethany,Betsabé,Bettina
6,Betânia,Beverly,Bibiana,Blanche,Bridget,Brigita,Brigite,Brunnah,Brígida,Bárbara,Báucide,Cacilda,Cacéres,Caillet,Caitlin,Caitria,Calista,Calíope,Camapuã,Cambuci
7,Cameron,Camille,Camilly,Camocim,Campbel,Camélia,Canirim,Caprina,Carlota,Carmela,Cassidy,Catrina,Cecília,Celeste,Cembira,Cendira,Ceressa,Chandra,Chelsea,Cimeína
8,Cipoúna,Clarice,Clarina,Clarita,Cleanto,Cláudia,Clíntia,Coaraci,Colares,Colomba,Cordula,Corália,Cristal,Cynthia,Cândida,Dagmara,Dakarai,Dalmira,Dalvina,Dalziza
9,Damiana,Dandara,Daniela,Daniele,Danúbia,Darlene,Deborah,Dejanir,Delamar,Delfina,Demeter,Destiny,Deífila,Diacira,Diadema,Diomara,Dionira,Djamila,Djamira,Dolores


In [0]:
def calculate_mean(alphabet, name):
    name = remover_acentos(name.lower())
    cname = list(name.replace(' ', ''))
    
    summ = 0
    for letter in alphabet:
        for l in cname:
            if letter.letter == l:
                summ += letter.num
                break
    return int(np.floor(summ / len(name.split())))

alphabet = []
letters = list(string.ascii_lowercase)
for i, l in enumerate(letters):
    alphabet.append(Letter(num=i+1, letter=l))

f1_mean = calculate_mean(alphabet, 'Rodolfo De Nadai')
f2_mean = calculate_mean(alphabet, 'Claudia Lazarini Meleti')

mi, mx = np.min([f1_mean, f2_mean]) - 1, np.max([f1_mean, f2_mean]) + 1

sobrenome = 'Lazarini Meleti De Nadai'
RM = []
for m in M:
    full_name = f'{m} {sobrenome}'
    mean_c = calculate_mean(alphabet, full_name)
    if mi < mean_c < mx:
        RM.append(m)
RF = []
for f in F:
    full_name = f'{f} {sobrenome}'
    mean_c = calculate_mean(alphabet, full_name)
    if mi < mean_c < mx:
        RF.append(f)

In [89]:
df = pd.DataFrame(side_by_side(RM, N), columns=['' for _ in range(N)])
df[df.isnull()] = ''
display(df)

df = pd.DataFrame(side_by_side(RF, N), columns=['' for _ in range(N)])
df[df.isnull()] = ''
display(df)

,,,,,,,,,,,,,,,,,,,,
0,Absalom,Absirto,Abúndio,Accioli,Acelino,Acemiro,Acilino,Acineto,Acrísio,Acúrcio,Acúrsio,Adelpho,Adelson,Aderico,Adonias,Adílson,Afrânio,Agasias,Aglaupo,Agledes
1,Agnaldo,Agrécio,Aguilar,Aguimar,Aguirre,Agustin,Ajagunã,Alapaki,Alarico,Alceste,Alcindo,Alcídes,Aldonca,Alexino,Alfonso,Alfredo,Allyson,Almasor,Aloísio,Aluísio
2,Amintas,Amorreu,Ampélio,Amâncio,Américo,Ananias,Aniceto,Ansaldo,Anselmo,Anthony,Antunes,Antíoco,Antíope,Apoinar,Apolino,Apollon,Aprígio,Aquiles,Aquilão,Aquílio
3,Arajuba,Arantes,Araquém,Araruna,Arcádio,Arduíno,Argaios,Ariosto,Aristeu,Arnolfo,Arnulfo,Arouche,Arsênio,Arvídio,Aríston,Ascânio,Asenate,Aspásio,Assuero,Astride
4,Astêmio,Ataulfo,Augusto,Aurélio,Avelino,Azarias,Azevedo,Bacuara,Baldwin,Balster,Barnaby,Barsaré,Baruque,Basileu,Basílio,Batimeu,Batista,Beaulah,Belquir,Belquis
5,Ben-Hur,Benigno,Benício,Birigui,Bismark,Bogdana,Bogumil,Bolívar,Bourbon,Briaréu,Bristol,Bronwen,Brígido,Burnier,Caetano,Calisto,Calixto,Calvino,Calídno,Camarão
6,Cardoso,Carioca,Carlito,Carlton,Carmelo,Caronte,Casimir,Castiel,Caxambu,Cecílio,Cercion,Charles,Chester,Cirineu,Ciríaco,Clayton,Cleiton,Cleonie,Cleonte,Cleyton
7,Cleófas,Clinton,Cláudio,Cláusio,Clâncio,Coimbra,Colombo,Conrado,Cordato,Corinto,Crispim,Crispin,Curvelo,Czeslaw,Cândido,Cândole,Cécrops,Daciano,Dagomar,Daltivo
8,Damásio,Darceno,Delfino,Dercino,Diaurum,Dickson,Diocles,Dominic,Domício,Dorival,Douglas,Drímaco,Durward,Dáscilo,Délfico,Edgardo,Edilson,Edmundo,Eduardo,Edvaldo
9,Egberto,Eistein,Ekemona,Ekewaka,Eliasoé,Elpenor,Elpídio,Emanuel,Emerson,Enrique,Ephraim,Epêneto,Epícuro,Epígone,Epíteto,Ercílio,Ernesto,Escobar,Esmondo,Estevam


,,,,,,,,,,,,,,,,,,,,
0,Aaliyah,Abayomi,Adônica,Agenora,Alawara,Alcione,Alekona,Aleluia,Alepeka,Alessia,Alisson,Amanacy,Amapola,Ambrose,Analisa,Andresa,Anelise,Anselma,Anselme,Anúncia
1,Apolina,Apoédia,Aretusa,Arsione,Ashiley,Aspásia,Assunta,Astéria,Augusta,Aurélia,Avelina,Azucena,Açucena,Baepeba,Baraúna,Barbosa,Belfort,Bersabé,Bethany,Betsabé
2,Beverly,Brunnah,Báucide,Cacéres,Calista,Calíope,Camapuã,Cambuci,Cameron,Camilly,Camocim,Campbel,Caprina,Carlota,Cassidy,Celeste,Ceressa,Chelsea,Cipoúna,Cleanto
3,Cláudia,Coaraci,Colares,Colomba,Cordula,Corália,Cristal,Cynthia,Dalvina,Danúbia,Deborah,Destiny,Dolores,Dominga,Dorothy,Dâmaris,Désirée,Ednalva,Eduarda,Edwiges
4,Eirapuã,Elisama,Esméria,Estácia,Eucaris,Eucléia,Eucádia,Eudócia,Eudóxia,Eufêmia,Eugênia,Eulália,Eurides,Euríale,Eusébia,Eutália,Evelina,Eveline,Evelise,Evellyn
5,Eyshila,Fabíola,Ferônia,Flórida,Frances,Francis,Geisila,Gelásia,Genevra,Genésia,Geovana,Getúlia,Geórgia,Giovana,Giselda,Giselle,Gislaim,Giullia,Giórgia,Glaucia
6,Guacira,Guapira,Guinete,Guiomar,Gwineth,Géssica,Hadassa,Haikela,Harpina,Heloise,Heloísa,Helvina,Hideaki,Horácia,Hélcias,Igapira,Ikepela,Isabela,Isabele,Isadora
7,Isidora,Isidore,Isméria,Ismênia,Isolina,Ivanete,Ivanice,Ivonete,Jamilly,Jasmine,Jathara,Jerusha,Jesebel,Jesuína,Joannes,Jocasta,Jordana,Joseane,Josiana,Josiane
8,Josélia,Juacira,Juanita,Juarana,Juatama,Jucélia,Juliana,Juliane,Julieta,Juliete,Jupiara,Jussara,Justina,Justine,Juvenca,Jéssica,Kamilly,Karolyn,Kaylane,Kemilly
9,Kerensa,Kerstin,Khadija,Krishna,Kristal,Kristen,Lakshmi,Lanassa,Larissa,Latasha,Laulena,Laureen,Laverna,Laverne,Lavínea,Lavínia,Lemuela,Levinda,Lisiane,Lourdes
